### Big data course project
<strong>T7: Forecasting demand for services separately</strong>

Jovana Videnovic & Haris Kupinic

In [ ]:
!hostnamectl

In [ ]:
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd
from pathlib import Path
import pandas as pd
import os
import numpy as np
from sklearn.neighbors import BallTree
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.compute as pc
from datetime import timedelta
from dask_ml.linear_model import LinearRegression
from dask_ml.ensemble import BlockwiseVotingRegressor
import dask.array as da
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from xgboost import dask as dxgb
import lightgbm as lgb

In [ ]:
def bootstrapping(errors, n_iterations=1000):
    """Perform bootstrapping to estimate confidence intervals."""
    n_size = len(errors)
    indices = np.random.randint(0, n_size, (n_iterations, n_size))
    samples = errors[indices]
    means = np.mean(samples, axis=1)
    lower = np.percentile(means, 2.5)
    upper = np.percentile(means, 97.5)
    return lower, upper

In [ ]:
cluster = LocalCluster(n_workers=2, threads_per_worker=2, memory_limit='64GB')
client = Client(cluster)

In [ ]:
data_path = Path("/d/hpc/home/jv8043/BD/project/T7/data")

In [ ]:
dc_y = pd.read_csv(data_path / "dc_y_augmented.csv")
dc_g = pd.read_csv(data_path / "dc_g_augmented.csv")
dc_fhv = pd.read_csv(data_path / "dc_fhv_augmented.csv")
dc_fhvhv = pd.read_csv(data_path / "dc_fhvhv_augmented.csv")

In [ ]:
# select only from 2019-03-01 to 2019-03-31
dc_y = dc_y[(dc_y["pickup_datetime"] >= "2019-03-01") & (dc_y["pickup_datetime"] < "2025-01-15")]
dc_g = dc_g[(dc_g["pickup_datetime"] >= "2019-03-01") & (dc_g["pickup_datetime"] < "2025-01-15")]
dc_fhv = dc_fhv[(dc_fhv["pickup_datetime"] >= "2019-03-01") & (dc_fhv["pickup_datetime"] < "2025-01-15")]
dc_fhvhv = dc_fhvhv[(dc_fhvhv["pickup_datetime"] >= "2019-03-01") & (dc_fhvhv["pickup_datetime"] < "2025-01-15")]

### Minimal setting

In [ ]:
def one_hot_encode(train_df, test_df, categorical_cols):
    train_encoded = pd.get_dummies(train_df, columns=categorical_cols, drop_first=True)
    test_encoded = pd.get_dummies(test_df, columns=categorical_cols, drop_first=True)
    test_encoded = test_encoded.reindex(columns=train_encoded.columns, fill_value=0)
    return train_encoded, test_encoded

In [ ]:
test_size = 14

In [ ]:
final_pred = np.zeros((test_size, 1))
final_gt = np.zeros((test_size, 1))

In [ ]:
for daily_counts, s_name in zip([dc_y, dc_g, dc_fhv, dc_fhvhv], ["dc_y", "dc_g", "dc_fhv", "dc_fhvhv"]):   
    # add season column
    def get_season(dt):
        month = dt.month
        if month in [12, 1, 2]:
            return 'winter'
        elif month in [3, 4, 5]:
            return 'spring'
        elif month in [6, 7, 8]:
            return 'summer'
        else:
            return 'autumn'

    daily_counts['pickup_datetime'] = pd.to_datetime(daily_counts['pickup_datetime'])
    daily_counts['season'] = daily_counts['pickup_datetime'].apply(get_season)
    daily_counts['day'] = daily_counts['pickup_datetime'].dt.day
    daily_counts['week_in_month'] = (daily_counts['pickup_datetime'].dt.day - 1) // 7 + 1
    daily_counts['day_of_year'] = daily_counts['pickup_datetime'].dt.dayofyear
    daily_counts['day_of_week'] = daily_counts['pickup_datetime'].dt.dayofweek
    daily_counts['year'] = daily_counts['pickup_datetime'].dt.year
    daily_counts['month'] = daily_counts['pickup_datetime'].dt.month
    daily_counts["weekend"] = daily_counts["day_of_week"].isin([5, 6]).astype(int)

    if "GB" not in MODEL_NAME:
        daily_counts["day_of_week_sin"] = np.sin(2 * np.pi * daily_counts["day_of_week"] / 7)
        daily_counts["day_of_week_cos"] = np.cos(2 * np.pi * daily_counts["day_of_week"] / 7)
        daily_counts["month_sin"] = np.sin(2 * np.pi * daily_counts["month"] / 12)
        daily_counts["month_cos"] = np.cos(2 * np.pi * daily_counts["month"] / 12)
        daily_counts["day_of_year_sin"] = np.sin(2 * np.pi * daily_counts["day_of_year"] / 365)
        daily_counts["day_of_year_cos"] = np.cos(2 * np.pi * daily_counts["day_of_year"] / 365)

        del daily_counts["month"]
        del daily_counts['year']  
    else:
        pass
        # from sklearn.preprocessing import LabelEncoder

        # # === 1. Define custom categorization functions for each feature ===

        # def categorize_awnd(val):  # Average wind speed
        #     if val < 4:
        #         return 'windy'
        #     else:
        #         return 'very_windy'

        # def categorize_prcp(val):  # Precipitation
        #     if val == 0:
        #         return 'none'
        #     else:
        #         return 'rain'
           
        # def categorize_snow(val):  # Snowfall
        #     if val == 0:
        #         return 'none'
        #     else:
        #         return 'snow'
            
        # def categorize_snwd(val):  # Snow depth
        #     if val == 0:
        #         return 'none'
        #     else:
        #         return 'snow'

        # def categorize_temp(temp):  # For TMAX and TMIN
        #     if temp < 0:
        #         return 'freezing'
        #     elif temp < 15:
        #         return 'cold'
        #     else:
        #         return 'hot'

        # # === 2. Apply categorizations ===

        # daily_counts['AWND_cat'] = daily_counts['AWND'].apply(categorize_awnd)
        # daily_counts['PRCP_cat'] = daily_counts['PRCP'].apply(categorize_prcp)
        # daily_counts['SNOW_cat'] = daily_counts['SNOW'].apply(categorize_snow)
        # daily_counts['SNWD_cat'] = daily_counts['SNWD'].apply(categorize_snwd)
        # daily_counts['TMAX_cat'] = daily_counts['TMAX'].apply(categorize_temp)
        # daily_counts['TMIN_cat'] = daily_counts['TMIN'].apply(categorize_temp)

        # # === 3. Encode categories with LabelEncoder ===

        # categorical_cols = ['AWND_cat', 'PRCP_cat', 'SNOW_cat', 'SNWD_cat', 'TMAX_cat', 'TMIN_cat']
        
        # for col in categorical_cols:
        #     le = LabelEncoder()
        #     encoded_col = col + '_enc'
        #     daily_counts[encoded_col] = le.fit_transform(daily_counts[col])
        
        # del daily_counts['AWND']
        # del daily_counts['PRCP']
        # del daily_counts['SNOW']
        # del daily_counts['SNWD']
        # del daily_counts['TMAX']
        # del daily_counts['TMIN']
        # del daily_counts['AWND_cat']
        # del daily_counts['PRCP_cat']
        # del daily_counts['SNOW_cat']
        # del daily_counts['SNWD_cat']
        # del daily_counts['TMAX_cat']
        # del daily_counts['TMIN_cat']

    print(daily_counts.columns)
    daily_counts = daily_counts[daily_counts['ride_count'] > 1000]
    del daily_counts["pickup_datetime"]
    del daily_counts["day_of_year"]
    del daily_counts["day_of_week"]

    train = daily_counts.iloc[:-test_size]
    test = daily_counts.iloc[-test_size:]
    if "GB" not in MODEL_NAME:
        train, test = one_hot_encode(train, test, ['season'])
    else:
        train, test = one_hot_encode(train, test, ['season', 'month', 'year'])
        
    cols = train.columns.difference(['ride_count', 'pickup_datetime'])

    for c in cols:
        train[c] = train[c].astype('float')
        test[c] = test[c].astype('float')

    # Convert to Dask arrays
    # select all but 'ride_count' and 'pickup_datetime'
    X_train = da.from_array(train[cols].values)
    y_train = da.from_array(train['ride_count'].values)

    X_test = da.from_array(test[cols].values)
    y_test = da.from_array(test['ride_count'].values)
    
    final_gt += y_test.compute().reshape(-1, 1)


    if MODEL_NAME == "DXGB":
        dtrain = dxgb.DaskDMatrix(client, X_train, y_train)
        output = dxgb.train(
            client,
            {"verbosity": 2, "tree_method": "hist", "objective": "reg:linear"},
            dtrain,
            num_boost_round=4,
            evals=[(dtrain, "train")],
    )    
        prediction = dxgb.predict(client, output, X_test)
        y_pred = prediction.compute()
    elif MODEL_NAME == "XGB":
        est = dxgb.XGBRegressor()
        est.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
        # Predict
        y_pred = est.predict(X_test)
    elif MODEL_NAME == "LGB":
        dask_model = lgb.DaskLGBMRegressor(client=client)
        dask_model = dask_model.fit(X_train, y_train)
        y_pred = dask_model.predict(X_test).compute()
    else:
        est = BlockwiseVotingRegressor(
            estimator= LinearRegression(),
        )
        # Fit the model
        est.fit(X_train, y_train)

        # Predict
        y_pred = est.predict(X_test).compute()

    final_pred += y_pred.reshape(-1, 1)

    # Calculate and print metrics
    absolute_errors = np.abs(y_test.compute() - y_pred)
    lower, upper = bootstrapping(absolute_errors)
    
    print(f"Confidence intervals for {s_name}:")
    print(f"Lower bound: {lower}")
    print(f"Upper bound: {upper}")
    print("MAE", np.mean(absolute_errors))

    mape_errors = np.abs((y_test.compute() - y_pred) / y_test.compute())
    mape_lower, mape_upper = bootstrapping(mape_errors)
    
    print(f"Confidence intervals for MAPE {s_name}:")
    print(f"Lower bound: {mape_lower}")
    print(f"Upper bound: {mape_upper}")
    print("MAPE", np.mean(mape_errors))

    print("-" * 40)

In [ ]:
absolute_errors = np.abs(final_gt - final_pred)
lower, upper = bootstrapping(absolute_errors)

print("Final confidence intervals:")
print(f"Lower bound: {lower}")
print(f"Upper bound: {upper}")
print("Final MAE", np.mean(absolute_errors))

print("-" * 40)
mape_errors = np.abs((final_gt - final_pred) / final_gt)
mape_lower, mape_upper = bootstrapping(mape_errors)

print("Final confidence intervals for MAPE:")
print(f"Lower bound: {mape_lower}")
print(f"Upper bound: {mape_upper}")
print("Final MAPE", np.mean(mape_errors))

In [ ]:
# mape
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(final_gt, final_pred) * 100
print(f"MAPE: {mape:.2f}%")